In [7]:
#Install tqdm to have progress bar
!pip install tqdm

!pip install requests
!pip install requests-aws4auth
!pip install "sagemaker>=2.0.0<3.0.0"

!pip install install transformers[tensorflow]
!pip install -q sagemaker-experiments==0.1.13


In [2]:
#Set up ipywigets 
#%%capture
import boto3
import sagemaker
from sagemaker import get_execution_role
import numpy as np
import json




In [3]:
#Get execution roles 
##The following line WILL NOT RUN outside of sagemaker notebook! It uses the notebook IAM to do deployments and invoke endpoints. 
##This can be replaced with other IAM role setups with boto3, but this is more involved and I cannot do it on infra I do not have acess to 
role = get_execution_role()

s3_resource = boto3.resource("s3")
s3 = boto3.client('s3')

print(f'SageMaker SDK Version: {sagemaker.__version__}')

SageMaker SDK Version: 2.109.0


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [4]:
#Defining default bucket for SageMaker pretrained model hosting
sagemaker_session = sagemaker.Session()

In [5]:
##########
##########

#Deploy documentation generation API 

In [9]:
from sagemaker.estimator import Estimator

hyperparameters = {"train-steps": 100}

#time.sleep(500)

estimator_unattatched = Estimator(
role=role,
instance_count=1,
instance_type="ml.c4.xlarge",
image_uri="708362204001.dkr.ecr.us-east-2.amazonaws.com/documentation-generation-api:latest",
hyperparameters=hyperparameters,
)


In [10]:
##Deploying model requires an attatched train job, 
##for first run just hack by running empty train script with random file 
estimator_unattatched.fit("<s3 path to a text file with anything in it>")

2023-01-10 20:28:20 Starting - Starting the training job...
2023-01-10 20:28:43 Starting - Preparing the instances for trainingProfilerReport-1673382500: InProgress
......
2023-01-10 20:29:43 Downloading - Downloading input data...
2023-01-10 20:30:04 Training - Downloading the training image......
2023-01-10 20:31:04 Training - Training image download completed. Training in progress..this is a training script!

2023-01-10 20:31:44 Uploading - Uploading generated training model
2023-01-10 20:31:44 Completed - Training job completed
Training seconds: 117
Billable seconds: 117


In [ ]:
###Only redeploy if new container
predictor = estimator.deploy(1, instance_type = "ml.g4dn.xlarge")

---------------------------